## Transforming into NER Dataset
Now we have the data into the format we want, we still need to put it into the format of an NER datast. We need it in the format IOB (Inside, Out, Beginning) format for each word. But first we need to create the tags that we want.

In [109]:
import json
import pandas as pd


In [160]:
charities_train = pd.read_csv("realKieData/charities/train.csv")
charities_validation = pd.read_csv("realKieData/charities/val.csv")
charities_test = pd.read_csv("realKieData/charities/test.csv")

nda_train = pd.read_csv("realKieData/nda/train.csv")
nda_validation = pd.read_csv("realKieData/nda/val.csv")
nda_test = pd.read_csv("realKieData/nda/test.csv")

fcc_train = pd.read_csv("realKieData/fcc/train.csv")
fcc_validation = pd.read_csv("realKieData/fcc/val.csv")
fcc_test = pd.read_csv("realKieData/fcc/test.csv")

rc_train = pd.read_csv("realKieData/rc/train.csv")
rc_validation = pd.read_csv("realKieData/rc/val.csv")
rc_test = pd.read_csv("realKieData/rc/test.csv")

s1_train = pd.read_csv("realKieData/s1/train.csv")
s1_validation = pd.read_csv("realKieData/s1/val.csv")
s1_test = pd.read_csv("realKieData/s1/test.csv")


In [161]:
charities_train['labels'] = charities_train['labels'].apply(json.loads)
charities_validation['labels'] = charities_validation['labels'].apply(json.loads)
charities_test['labels'] = charities_test['labels'].apply(json.loads)

nda_train['labels'] = nda_train['labels'].apply(json.loads)
nda_validation['labels'] = nda_validation['labels'].apply(json.loads)
nda_test['labels'] = nda_test['labels'].apply(json.loads)

fcc_train['labels'] = fcc_train['labels'].apply(json.loads)
fcc_validation['labels'] = fcc_validation['labels'].apply(json.loads)
fcc_test['labels'] = fcc_test['labels'].apply(json.loads)

rc_train['labels'] = rc_train['labels'].apply(json.loads)
rc_validation['labels'] = rc_validation['labels'].apply(json.loads)
rc_test['labels'] = rc_test['labels'].apply(json.loads)

s1_train['labels'] = s1_train['labels'].apply(json.loads)
s1_validation['labels'] = s1_validation['labels'].apply(json.loads)
s1_test['labels'] = s1_test['labels'].apply(json.loads)

In [162]:
charities_train.iloc[0]['labels'][0]


{'label': 'Charity Name',
 'start': 2950,
 'end': 2970,
 'text': 'MICKLEHAM ALMSHOUSES'}

In [163]:
charities_train.text[0][2950:2970]

'MICKLEHAM ALMSHOUSES'

## Investigating different entities 

In [164]:
from collections import Counter
def count_labels(dataset, column_name='labels'):
    all_labels = []
    
    # Iterate over each row and parse the JSON-like strings
    for row in dataset[column_name]:
        try:
            # Parse the JSON-like string
            # Extract labels and add to the all_labels list
            for item in row:
                all_labels.append(item['label'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in row: {row} - {e}")
    
    # Count the occurrences of each label
    label_counts = Counter(all_labels)
    
    # Print the counts
    for label, count in label_counts.items():
        print(f"{label}: {count}")

In [165]:
count_labels(charities_train)

Charity Name: 4250
Charity Registered Number: 729
Year Ended: 3644
Accounting Basis: 231
Independent Examiner Name: 398
Examination Date: 211
Independent Examiner Street Address: 534
Independent Examiner City: 549
Independent Examiner Postal Code: 515
Net Income at Current Year End: 162
Net Income at Previous Year End: 165
Net Assets at Current Year End: 242
Net Assets at Previous Year End: 231
Cash In Hand at Current Year End: 281
Cash In Hand at Previous Year End: 273
Principal Office Street Address: 294
Principal Office City: 295
Principal Office Postal Code: 283
Trustee Name: 3510
Trustee Title: 1051
Objectives and Activities: 270
Independent Examiner Company: 447
Bank Name: 233
Named Donor: 445
Company Number: 181
Named Employee: 52
Event Name: 136
Project Name: 107


In [166]:
count_labels(nda_train)

Jurisdiction: 243
Party: 522
Effective Date: 237


In [167]:
count_labels(fcc_train)

Line Item - Description: 10170
Line Item - Days: 8810
Line Item - Rate: 11980
Line Item - Start Date: 11889
Line Item - End Date: 6010
Agency: 399
Advertiser: 608
Gross Total: 496
Net Amount Due: 375
Agency Commission: 224
Payment Terms: 254


In [168]:
count_labels(rc_train)

(Header) Contract Area Description: 144
Contract Area Description: 574
(Header)Governing law: 140
Governing law: 143
(Header)Hardship clause or force majeure: 118
Hardship clause or force majeure: 158
(Header)Reporting requirements: 301
Reporting requirements: 762
(Header)Environmental protections: 124
Environmental protections: 296
(Header)Income tax: rate: 101
Income tax: rate: 101
(Header)Term: 165
Term: 217
Renewal or extension of term: 271
Type: 202
Date Signed: 171
Participants: 518
Country: 243
Project: 117
Water use: 72
Signatories, company: 283
(Header)Water use: 19


In [169]:
count_labels(s1_train)

Company Officer: 1528
Company Officer Title: 1547
Risk Clauses: 14487
(Header) Risks To The Business: 194
(Header) Description of Securities: 220
Description of Securities (1st Para): 226
(Header) Dividend Policy: 180
Dividend Policy (1st Para): 181
(Header) Prospectus Summary: 191
Prospectus Summary (1st Para): 1852
Joint Book Runners: 352
Title of Security Registered: 537
Amount Registered: 524
Max Price: 280
Date of Prospectus: 187
Company Name: 195
Company Address: 192
Agent Name: 190
Agent Address: 190
Agent Telephone: 183
EIN: 187
Attorney Names: 738
Law Firm Name: 382
Law Firm Address: 527


# Cleaning text, removing blank lines and adjusting offsets.

In [170]:
def remove_empty_labels(df, dataset_name, split):
    # Filter out rows where labels are equal to '[]' and print them
    empty_labels_rows = df[df['labels'] == '[]']
    if not empty_labels_rows.empty:
        print(f"Removed rows from {dataset_name} {split}:")
        #print(empty_labels_rows)
    
    return df[df['labels'] != '[]']

In [171]:
charities_train = remove_empty_labels(charities_train, 'charities', 'train')
charities_validation = remove_empty_labels(charities_validation, 'charities', 'validation')
charities_test = remove_empty_labels(charities_test, 'charities', 'test')

nda_train = remove_empty_labels(nda_train, 'nda', 'train')
nda_validation = remove_empty_labels(nda_validation, 'nda', 'validation')
nda_test = remove_empty_labels(nda_test, 'nda', 'test')

fcc_train = remove_empty_labels(fcc_train, 'fcc', 'train')
fcc_validation = remove_empty_labels(fcc_validation, 'fcc', 'validation')
fcc_test = remove_empty_labels(fcc_test, 'fcc', 'test')

rc_train = remove_empty_labels(rc_train, 'rc', 'train')
rc_validation = remove_empty_labels(rc_validation, 'rc', 'validation')
rc_test = remove_empty_labels(rc_test, 'rc', 'test')

s1_train = remove_empty_labels(s1_train, 's1', 'train')
s1_validation = remove_empty_labels(s1_validation, 's1', 'validation')
s1_test = remove_empty_labels(s1_test, 's1', 'test')

In [172]:
#drop all columns apart from text and labels
charities_train = charities_train[['text', 'labels']]
charities_validation = charities_validation[['text', 'labels']]
charities_test = charities_test[['text', 'labels']]

nda_train = nda_train[['text', 'labels']]
nda_validation = nda_validation[['text', 'labels']]
nda_test = nda_test[['text', 'labels']]

fcc_train = fcc_train[['text', 'labels']]
fcc_validation = fcc_validation[['text', 'labels']]
fcc_test = fcc_test[['text', 'labels']]

rc_train = rc_train[['text', 'labels']]
rc_validation = rc_validation[['text', 'labels']]
rc_test = rc_test[['text', 'labels']]

s1_train = s1_train[['text', 'labels']]
s1_validation = s1_validation[['text', 'labels']]
s1_test = s1_test[['text', 'labels']]

In [127]:
import re

#Not getting used
def clean_text_with_offsets(text):
    """
    Clean the text by replacing multiple spaces with a single space
    and update the offsets accordingly.
    """
    clean_text = []
    offset_map = []
    current_offset = 0
    
    # Replace multiple spaces with a single space
    for match in re.finditer(r'\S+|\s+', text):
        token = match.group(0)
        if token.isspace():
            if clean_text and clean_text[-1] != ' ':
                clean_text.append(' ')
                offset_map.append(current_offset)
        else:
            clean_text.append(token)
            offset_map.extend([current_offset] * len(token))
        current_offset += len(token)
    
    return ''.join(clean_text).strip(), offset_map


text = "hello    h"

cleaned_text, offset_map = clean_text_with_offsets(text)

print("Original text:", text)
print("Cleaned text:", cleaned_text)
print("Offset map:", offset_map)


Original text: hello    h
Cleaned text: hello h
Offset map: [0, 0, 0, 0, 0, 5, 9]


# Create the entities 
We only use some of the labels to create 4 entities for the model initally. PER (Person), ORG (Orginisation), LOC (Location), FIN (Finance - monetary amounts)

Need to make the label to entity mappings

In [173]:
label_to_entity = {
    "Charity Name": "ORG",
    "Bank Name": "ORG",
    "Party": "ORG",
    "Agency": "ORG",
    "Participants": "ORG",
    "Law Firm Name": "ORG",
    "Company Name": "ORG",
    "Joint Book Runners": "ORG",
    "Company Officer": "PER",
    "Attorney names" : "PER",
    "Independent Examiner Name": "PER",
    "Trustee Name": "PER",
    "Signatories": "PER",
    "Company": "PER",
    "Date Signed": "DAT",
    "Effective Date": "DAT",
    "Examination Date": "DAT",
    "Year Ended": "DAT",
    "Date of Prospectus": "DAT",
    "Line Item - Start Date": "DAT",
    "Line Item - End Date": "DAT",
    "Independent Examiner City": "LOC",
    "Principal Office City": "LOC",
    "Jurisdiction": "LOC",
    "Country": "LOC",
    "Governing Law": "LOC",
    "Net Income at Current Year End": "FIN",
    "Net Income at Previous Year End": "FIN",
    "Net Assets at Current Year End": "FIN",
    "Net Assets at Previous Year End": "FIN",
    "Cash In Hand at Current Year End": "FIN",
    "Cash In Hand at Previous Year End": "FIN",
    "Max Price": "FIN",
    "Net Amount Due": "FIN",
    "Gross Total": "FIN"
}

In [183]:
import pandas as pd
from transformers import BertTokenizerFast

#Using BERT fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


def label_tokens(labels, tokenized_inputs, token_labels):
    for label in labels:
        start_char = label['start']
        end_char = label['end']
        label_text = label['label']
        
        if label_text in label_to_entity:
            entity = label_to_entity[label_text]
            
            for idx, (start, end) in enumerate(tokenized_inputs['offset_mapping']):
                if start >= start_char and end <= end_char:
                    # Assign B-<entity> for the beginning token and I-<entity> for inside tokens
                    if start == start_char:
                        token_labels[idx] = f"B-{entity}"
                    else:
                        token_labels[idx] = f"I-{entity}"
                    
    return token_labels

def process_dataframe(dataframe):
    results = []

    for index, row in dataframe.iterrows():
        passage_id = index  # Using the index as the passage ID
        text = row['text']
        labels = row['labels']
        
        tokenized_inputs = tokenizer(text, return_offsets_mapping=True, truncation=True, max_length=512)
        
        token_labels = ["O"] * len(tokenized_inputs['input_ids'])
        
        token_labels = label_tokens(labels, tokenized_inputs, token_labels)
        
        for token, label in zip(tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids']), token_labels):
            results.append({
                'Passage_ID': passage_id,
                'Token': token,
                'Label': label
            })

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)
    
    return results_df



In [192]:
charities_test_labeled = process_dataframe(charities_test)
charities_train_labeled = process_dataframe(charities_train)
charities_validation_labeled = process_dataframe(charities_validation)

nda_test_labeled = process_dataframe(nda_test)
nda_train_labeled = process_dataframe(nda_train)
nda_validation_labeled = process_dataframe(nda_validation)

fcc_test_labeled = process_dataframe(fcc_test)
fcc_train_labeled = process_dataframe(fcc_train)
fcc_validation_labeled = process_dataframe(fcc_validation)

rc_test_labeled = process_dataframe(rc_test)
rc_train_labeled = process_dataframe(rc_train)
rc_validation_labeled = process_dataframe(rc_validation)

s1_test_labeled = process_dataframe(s1_test)
s1_train_labeled = process_dataframe(s1_train)
s1_validation_labeled = process_dataframe(s1_validation)

In [186]:
charities_train_labeled = process_dataframe(charities_train)

In [188]:
charities_train_labeled

,Passage_ID,Token,Label
0,0,[CLS],B-ORG
1,0,mick,B-ORG
2,0,##le,I-ORG
3,0,##ham,I-ORG
4,0,al,I-ORG
...,...,...,...
163385,321,ms,O
163386,321,.,O
163387,321,diane,O
163388,321,ru,O


In [194]:
s1_train_labeled.Label.value_counts()

O        93942
I-ORG     3035
B-ORG      576
I-DAT      565
B-DAT      184
B-PER        1
I-PER        1
Name: Label, dtype: int64

# Next Step: Split it into sentences with berts context length.

In [196]:
import pandas as pd
from transformers import BertTokenizerFast

# Initialize the tokenize
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def group_tokens_to_sentences(df, max_length=512):
    sentences = []
    labels = []
    current_sentence = []
    current_labels = []
    current_length = 0

    for _, row in df.iterrows():
        token = row['Token']
        label = row['Label']
        
        token_length = len(tokenizer.encode(token, add_special_tokens=False))
        
        if current_length + token_length + 2 > max_length:
            sentences.append(current_sentence)
            labels.append(current_labels)
            current_sentence = []
            current_labels = []
            current_length = 0
        
        current_sentence.append(token)
        current_labels.append(label)
        current_length += token_length
    
    if current_sentence:
        sentences.append(current_sentence)
        labels.append(current_labels)
    
    return sentences, labels

sentences, sentence_labels = group_tokens_to_sentences(charities_test_labeled)

sentence_df = pd.DataFrame({
    'Sentence': sentences,
    'Labels': sentence_labels
})

# Display the new DataFrame
sentence_df


,Sentence,Labels
0,"[[CLS], charity, commission, number, -, 2004, ...","[O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-O..."
1,"[governance, and, management, the, charity, wa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[charity, during, the, period, were, as, follo...","[O, O, O, O, O, O, O, O, B-PER, I-PER, I-PER, ..."
3,"[##hea, ##th, village, hall, for, use, by, loc...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[vanessa, big, ##gs, (, from, 2nd, april, 2019...","[B-PER, I-PER, I-PER, O, O, O, O, O, O, O, O, ..."
...,...,...
124,"[as, our, chairman, mentioned, in, last, year,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,"[auditor, ##s, 13, to, 14, statement, of, fina...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
126,"[trustees, stephanie, owen, joanna, philip, ma...","[O, B-PER, I-PER, B-PER, I-PER, B-PER, I-PER, ..."
127,"[when, we, said, farewell, to, our, children, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
